In [1]:
import sys

PWD = '../../..'
sys.path.append(PWD)

import numpy as np
import pandas as pd

import tensorflow as tf

tf.get_logger().setLevel('ERROR')
from tensorflow import keras
from tensorflow.keras import optimizers
from tensorflow.keras import callbacks
import keras_tuner as kt

from src.network_utils import count_params
from src.cross_validator import KerasTunerCrossValidator
from src.dataset import load_dataset_train_test, load_dataset_train_val
from src.models import convnet_builder as bare_model_builder

In [2]:
PROJECT_NAME = 'convnet'

PLANE = 2
CHANNEL = 11

OVERWRITE = False

DATASET_PATH = PWD + '/data/dataset/dataset.pkl'
TRIALS_DIR = PWD + f'/data/model_selection/channel_{PLANE}_{CHANNEL}/tuner'
CROSSVAL_DIR = PWD + f'/data/model_selection/channel_{PLANE}_{CHANNEL}/cross_val'

LR = 0.01
ES_MIN_DELTA = 0.01

N_EPOCHS = 3000
BATCH_SIZE = 2048
MAX_TRIALS = 40
EXECUTIONS_PER_TRIAL = 2

TOP_N = 5
CROSSVAL_N_CV = 5
CROSSVAL_N_EXEC = 2
LOSS_WEIGHT = 1000

In [3]:
X_base_train, _, y_base_train, _ = load_dataset_train_test(PWD, PLANE, CHANNEL)
X_train, X_val, y_train, y_val = load_dataset_train_val(PWD, PLANE, CHANNEL)

X_base_train.shape, X_train.shape, X_val.shape

((22412, 24), (17929, 24), (4483, 24))

# Model

In [4]:
def model_builder(hp: kt.HyperParameters) -> keras.Model:
    # Convolutional network params
    hp_n_conv_blocks = hp.Int("n_conv_blocks", min_value=1, max_value=3, step=1)
    hp_n_conv_layers = hp.Int("n_conv_layers", min_value=1, max_value=3, step=1)
    hp_filters_mult = hp.Choice("conv_filters_mult", values=[1, 2, 4, 8])
    hp_conv_spatial_dropout = hp.Choice("conv_spatial_dropout", values=[0.0, 0.1, 0.2])

    # MLP at the end params
    hp_mlp_n_hidden_layers = hp.Int("n_mlp_hidden_layers", min_value=0, max_value=3, step=1, default=0)
    hp_mlp_units_mult, hp_mlp_dropout = None, None
    if hp_mlp_n_hidden_layers > 0:
        hp_mlp_units_mult = hp.Choice("mlp_units_mult", values=[1, 2, 4, 8, 16], default=4)
        hp_mlp_dropout = hp.Choice("mlp_dropout", values=[0.0, 0.2, 0.5])

    # Other params
    hp_batch_normalization = hp.Boolean("batch_normalization", default=False)
    hp_input_batch_normalization = hp.Boolean("input_batch_normalization", default=False)
    hp_normalize_signal = hp.Boolean("normalize_signal", default=False)

    model = bare_model_builder(hp_n_conv_blocks, hp_n_conv_layers, hp_filters_mult, hp_conv_spatial_dropout,
                               hp_mlp_n_hidden_layers, hp_mlp_units_mult, hp_mlp_dropout, hp_batch_normalization,
                               hp_input_batch_normalization, hp_normalize_signal)
    model.compile(loss='mse', optimizer=optimizers.Adam(LR), loss_weights=LOSS_WEIGHT)
    return model

In [5]:
model_builder(kt.HyperParameters()).summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 reshape (Reshape)           (None, 24, 1)             0         
                                                                 
 conv1d (Conv1D)             (None, 24, 8)             32        
                                                                 
 max_pooling1d (MaxPooling1D  (None, 12, 8)            0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 96)                0         
                                                                 
 dense (Dense)               (None, 1)                 97        
                                                                 
Total params: 129
Trainable params: 129
Non-trainable params: 0
__________________________________________________________

In [6]:
model_callbacks = [
    callbacks.EarlyStopping(patience=60, min_delta=ES_MIN_DELTA),
    callbacks.ReduceLROnPlateau(monitor='loss', factor=0.9, patience=10)
]

# Bayesian tuner

In [7]:
bayesian_tuner = kt.BayesianOptimization(model_builder, objective='val_loss', executions_per_trial=EXECUTIONS_PER_TRIAL,
                                         max_trials=MAX_TRIALS, directory=TRIALS_DIR, project_name=PROJECT_NAME,
                                         overwrite=OVERWRITE)

bayesian_tuner.search(X_train, y_train, validation_data=[X_val, y_val], epochs=N_EPOCHS, callbacks=model_callbacks,
                      batch_size=BATCH_SIZE, verbose=3)

# Results

In [8]:
for i, hyperparameters in enumerate(bayesian_tuner.get_best_hyperparameters(TOP_N)):
    print(f'========== Model {i} ==========')
    print(hyperparameters.get_config()['values'])
    model_tmp = model_builder(hyperparameters)
    print('Number of parameters:', count_params(model_tmp))

========== Model 0 ==========
{'n_conv_blocks': 1, 'n_conv_layers': 1, 'conv_filters_mult': 1, 'conv_spatial_dropout': 0.0, 'n_mlp_hidden_layers': 3, 'batch_normalization': True, 'input_batch_normalization': True, 'normalize_signal': False, 'mlp_units_mult': 16, 'mlp_dropout': 0.5}
Number of parameters: 68001
========== Model 1 ==========
{'n_conv_blocks': 1, 'n_conv_layers': 3, 'conv_filters_mult': 2, 'conv_spatial_dropout': 0.0, 'n_mlp_hidden_layers': 0, 'batch_normalization': False, 'input_batch_normalization': True, 'normalize_signal': False, 'mlp_units_mult': 8, 'mlp_dropout': 0.5}
Number of parameters: 1921
========== Model 2 ==========
{'n_conv_blocks': 1, 'n_conv_layers': 2, 'conv_filters_mult': 8, 'conv_spatial_dropout': 0.2, 'n_mlp_hidden_layers': 3, 'batch_normalization': False, 'input_batch_normalization': False, 'normalize_signal': False, 'mlp_units_mult': 2, 'mlp_dropout': 0.0}
Number of parameters: 37889
========== Model 3 ==========
{'n_conv_blocks': 2, 'n_conv_layers':

# Cross-validation for top 5 models

In [9]:
cross_validator = KerasTunerCrossValidator(bayesian_tuner, X_base_train, y_base_train, model_builder,
                                           directory=CROSSVAL_DIR, project_name=PROJECT_NAME,
                                           n_epochs=N_EPOCHS, batch_size=BATCH_SIZE, n_top=TOP_N,
                                           es_min_delta=ES_MIN_DELTA, n_cv=CROSSVAL_N_CV, n_executions=CROSSVAL_N_EXEC,
                                           overwrite=OVERWRITE)
model_scores = cross_validator()

{'n_conv_blocks': 1, 'n_conv_layers': 1, 'conv_filters_mult': 1, 'conv_spatial_dropout': 0.0, 'n_mlp_hidden_layers': 3, 'batch_normalization': True, 'input_batch_normalization': True, 'normalize_signal': False, 'mlp_units_mult': 16, 'mlp_dropout': 0.5}
Number of parameters: 68001
Got score: 8.7402 (8.7650, 8.7155)
Got score: 8.4756 (8.5962, 8.3551)
Got score: 8.3256 (8.3760, 8.2752)
Got score: 8.6123 (8.6478, 8.5768)
Got score: 8.9057 (8.8743, 8.9371)


{'n_conv_blocks': 1, 'n_conv_layers': 3, 'conv_filters_mult': 2, 'conv_spatial_dropout': 0.0, 'n_mlp_hidden_layers': 0, 'batch_normalization': False, 'input_batch_normalization': True, 'normalize_signal': False, 'mlp_units_mult': 8, 'mlp_dropout': 0.5}
Number of parameters: 1921
Got score: 9.0949 (8.7066, 9.4832)
Got score: 8.5006 (8.4546, 8.5465)
Got score: 8.2513 (8.2543, 8.2482)
Got score: 8.6285 (8.7160, 8.5410)
Got score: 8.9366 (8.9617, 8.9114)


{'n_conv_blocks': 1, 'n_conv_layers': 2, 'conv_filters_mult': 8, 'conv_spatial_dropout': 0.2, 'n_mlp_hidden_layers': 3, 'batch_normalization': False, 'input_batch_normalization': False, 'normalize_signal': False, 'mlp_units_mult': 2, 'mlp_dropout': 0.0}
Number of parameters: 37889
Got score: 9.0500 (9.2962, 8.8038)
Got score: 16.0243 (17.9504, 14.0982)
Got score: 8.8217 (8.8983, 8.7452)
Got score: 11.7055 (8.8412, 14.5699)
Got score: 9.7360 (9.1258, 10.3463)


{'n_conv_blocks': 2, 'n_conv_layers': 1, 'conv_filters_mult': 8, 'conv_spatial_dropout': 0.1, 'n_mlp_hidden_layers': 0, 'batch_normalization': False, 'input_batch_normalization': False, 'normalize_signal': False, 'mlp_units_mult': 4, 'mlp_dropout': 0.5}
Number of parameters: 25729
Got score: 8.8281 (8.9187, 8.7375)
Got score: 8.4746 (8.4815, 8.4677)
Got score: 8.2900 (8.2953, 8.2847)
Got score: 8.6383 (8.7222, 8.5543)
Got score: 9.1201 (9.1879, 9.0523)


{'n_conv_blocks': 3, 'n_conv_layers': 3, 'conv_filters_mult': 4, 'conv_spatial_dropout': 0.1, 'n_mlp_hidden_layers': 2, 'batch_normalization': True, 'input_batch_normalization': True, 'normalize_signal': False, 'mlp_units_mult': 2, 'mlp_dropout': 0.2}
Number of parameters: 169697
Got score: 8.9489 (8.9664, 8.9315)
Got score: 8.5657 (8.5100, 8.6214)
Got score: 8.4334 (8.3788, 8.4879)
Got score: 8.5062 (8.5077, 8.5046)
Got score: 9.1907 (9.2662, 9.1152)


In [10]:
mean_scores = [f"{np.mean(scores):0.2f}" for scores in model_scores.values()]
std_scores = [f"{np.std(scores):0.2f}" for scores in model_scores.values()]
n_params = [count_params(model_builder(hyperparameters)) for hyperparameters in
            bayesian_tuner.get_best_hyperparameters(TOP_N)]

df = pd.DataFrame({'mean': mean_scores, 'std': std_scores, 'n_params': n_params}, index=model_scores.keys())
df.index.name = 'Model'
df

,mean,std,n_params
Model,,,
0,8.61,0.20,68001
1,8.68,0.30,1921
2,11.07,2.68,37889
3,8.67,0.29,25729
4,8.73,0.29,169697
